In [12]:
import glob
import logging
import os
import json
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from callback.optimizater.adamw import AdamW
from callback.lr_scheduler import get_linear_schedule_with_warmup
from callback.progressbar import ProgressBar
from tools.common import seed_everything, json_to_text
from tools.common import init_logger, logger

from models.transformers import WEIGHTS_NAME, BertConfig, AlbertConfig
from models.bert_for_ner import BertCrfForNer
from models.albert_for_ner import AlbertCrfForNer
from processors.utils_ner import CNerTokenizer, get_entities
from processors.ner_seq import convert_examples_to_features
from processors.ner_seq import ner_processors as processors
from processors.ner_seq import collate_fn
from metrics.ner_metrics import SeqEntityScore
from tools.finetuning_argparse import get_argparse

In [13]:
MODEL_CLASSES = {
    ## bert ernie bert_wwm bert_wwwm_ext
    'bert': (BertConfig, BertCrfForNer, CNerTokenizer),
    'albert': (AlbertConfig, AlbertCrfForNer, CNerTokenizer)
}

In [14]:
def load_and_cache_examples(args, task, tokenizer, data_type='train'):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
    processor = processors[task]()
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'cached_crf-{}_{}_{}_{}'.format(
        data_type,
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.train_max_seq_length if data_type == 'train' else args.eval_max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if data_type == 'train':
            examples = processor.get_train_examples(args.data_dir)
        elif data_type == 'dev':
            examples = processor.get_dev_examples(args.data_dir)
        else:
            examples = processor.get_test_examples(args.data_dir)
        features = convert_examples_to_features(examples=examples,
                                                tokenizer=tokenizer,
                                                label_list=label_list,
                                                max_seq_length=args.train_max_seq_length if data_type == 'train' \
                                                    else args.eval_max_seq_length,
                                                cls_token_at_end=bool(args.model_type in ["xlnet"]),
                                                pad_on_left=bool(args.model_type in ['xlnet']),
                                                cls_token=tokenizer.cls_token,
                                                cls_token_segment_id=2 if args.model_type in ["xlnet"] else 0,
                                                sep_token=tokenizer.sep_token,
                                                # pad on the left for xlnet
                                                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0,
                                                )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)
    all_lens = torch.tensor([f.input_len for f in features], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_lens, all_label_ids)
    return dataset

In [16]:
    output_dir = "outputs/skillner_output/bert/"
    task_name = "skillner"
    model_type = "bert"
    no_cuda = True
    local_rank = -1
    seed = 42
    
    
    adam_epsilon=1e-08
    adv_epsilon=1.0
    adv_name='word_embeddings'
    cache_dir=''
    config_name=''
    crf_learning_rate=5e-05
    data_dir='/Users/junix/code/CLUENER2020/pytorch_version/datasets/skillner/'
    do_adv=False
    do_eval=True
    do_lower_case=True
    do_predict=False
    do_train=False
    eval_all_checkpoints=False
    eval_max_seq_length=512
    evaluate_during_training=False
    fp16=False
    fp16_opt_level='O1'
    gradient_accumulation_steps=1
    learning_rate=3e-05
    local_rank=-1
    logging_steps=448
    loss_type='ce'
    markup='bios'
    max_grad_norm=1.0
    max_steps=-1
    model_name_or_path='/Users/junix/code/CLUENER2020/pytorch_version/prev_trained_model/roberta_wwm_large_ext'
    model_type='bert'
    no_cuda=False
    num_train_epochs=5.0
    output_dir='/Users/junix/code/CLUENER2020/pytorch_version/outputs/skillner_output/'
    overwrite_cache=False
    overwrite_output_dir=True
    per_gpu_eval_batch_size=24
    per_gpu_train_batch_size=24
    predict_checkpoints=0
    save_steps=448
    seed=42
    server_ip=''
    server_port=''
    task_name='skillner'
    tokenizer_name=''
    train_max_seq_length=128
    warmup_proportion=0.1
    weight_decay=0.01
    
    output_dir = output_dir + '{}'.format(model_type)

    
    time_ = time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime())
    init_logger(log_file=output_dir + f'/{model_type}-{task_name}-{time_}.log')

 
    # Setup CUDA, GPU & distributed training
    if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(local_rank)
        device = torch.device("cuda", local_rank)
        n_gpu = 1
        
    print(device, n_gpu)
    
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training",
        local_rank, device, n_gpu, bool(local_rank != -1),  )
    # Set seed
    seed_everything(seed)
    # Prepare NER task
 
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
        
    processor = processors[task_name]()
    label_list = processor.get_labels()
    id2label = {i: label for i, label in enumerate(label_list)}
    label2id = {label: i for i, label in enumerate(label_list)}
    num_labels = len(label_list)

    print(id2label)
    print(label2id)
    
    # Load pretrained model and tokenizer
    if local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    model_type = model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
    config = config_class.from_pretrained(config_name if config_name else model_name_or_path,
                                          num_labels=num_labels, cache_dir=cache_dir if cache_dir else None, )
    tokenizer = tokenizer_class.from_pretrained(tokenizer_name if tokenizer_name else model_name_or_path,
                                                do_lower_case=do_lower_case,
                                                cache_dir=cache_dir if cache_dir else None, )
    model = model_class.from_pretrained(model_name_or_path, from_tf=bool(".ckpt" in model_name_or_path),
                                        config=config, cache_dir=cache_dir if cache_dir else None)
    if local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model.to(device)
    logger.info("Training/evaluation parameters")
    
    print(tokenizer)

10/15/2021 09:57:28 - WARNING - root -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training
10/15/2021 09:57:28 - INFO - models.transformers.configuration_utils -   loading configuration file /Users/junix/code/CLUENER2020/pytorch_version/prev_trained_model/roberta_wwm_large_ext/config.json
10/15/2021 09:57:28 - INFO - models.transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "lstm_dropout_prob": 0.5,
  "lstm_embedding_size": 768,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc

cpu 0
{0: 'X', 1: 'B-skill', 2: 'I-skill', 3: 'S-skill', 4: 'O', 5: '[START]', 6: '[END]'}
{'X': 0, 'B-skill': 1, 'I-skill': 2, 'S-skill': 3, 'O': 4, '[START]': 5, '[END]': 6}


10/15/2021 09:57:31 - INFO - models.transformers.modeling_utils -   Weights of BertCrfForNer not initialized from pretrained model: ['classifier.weight', 'classifier.bias', 'crf.start_transitions', 'crf.end_transitions', 'crf.transitions']
10/15/2021 09:57:31 - INFO - models.transformers.modeling_utils -   Weights from pretrained model not used in BertCrfForNer: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
10/15/2021 09:57:31 - INFO - root -   Training/evaluation parameters


In [17]:

input_example = InputExample()

NameError: name 'InputExample' is not defined